In [8]:
import pandas as pd
import string
import math 
import numpy as np 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
pd.set_option('display.max_colwidth', 0)
df = pd.read_csv("publications_data.csv")
df.head()

,name,job_title,organization,doc_link,doc_title,doc_abstract
0,Mohamad Nazri Abd Karim,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/stock-price-and-volume-effects-associated-with-changes-in-the-com,Stock Price and Volume Effects Associated with Changes in the Composition of the FTSE Bursa Malaysian KLCI,"We examine the stock price and volume effects of changes in the composition of the FTSE Bursa Malaysia Kuala Lumpur Composite Index (KLCI), over the time period of 2005 to 2012. We find evidence to support the price pressure hypothesis for both additions to and deletions from the KLCI. This is because substantial stock price and trading volume effects in the pre index revision period are entirely reversed after the announcement of the news. Significant changes in liquidity causes trading volume and stock prices to reverse back to their original level before the index revisions took place."
1,Muhammad Abdullah,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/assessing-the-use-of-gold-as-a-zero-beta-asset-in-empirical-asset,Assessing the Use of Gold as a Zero-Beta Asset in Empirical Asset Pricing: Application to the US Equity Market,"This paper examines the use of the return on gold instead of treasury bills in empirical asset pricing models for the US equity market. It builds upon previous research on the safe-haven, hedging, and zero-beta characteristics of gold in developed markets and the close relationship between interest rates, stock, and gold returns. In particular, we extend this research by showing that using gold as a zero-beta asset helps to improve the time-series performance of asset pricing models when pricing US equities and industries between 1981 and 2015. The performance of gold zero-beta models is also compared with traditional empirical factor models using the 1-month Treasury bill rate as the risk-free rate. Our results indicate that using gold as a zero-beta asset leads to higher R-squared values, lower Sharpe ratios of alphas, and fewer significant pricing errors in the time-series analysis. Similarly, the pricing of small stock and industry portfolios is improved. In cross-section, we also find improved results, with fewer cross-sectional pricing errors and more economically meaningful pricing of risk factors. We also find that a zero-beta gold factor constructed to be orthogonal to the Carhart four factors is significant in cross-section and helps to improve factor model performance on momentum portfolios. Furthermore, the Fama–French three- and five-factor asset pricing models and the Carhart model are all improved by these means, particularly on test assets which have been poorly priced by the traditional versions. Our results have salient implications for policymakers, governments, central bank rate-setting decisions, and investors."
2,Al-Noor Abdullah,Lecturer in Business Management,School of Strategy and Leadership,https://pureportal.coventry.ac.uk/en/publications/social-impacts-of-a-mega-dam-project-as-perceived-by-local-resett,"Social Impacts of a Mega-Dam Project as Perceived by Local, Resettled and Displaced Communities: A Case Study of Merowe Dam, Sudan","The paper assesses social impacts of a mega-dam project (Merowe Dam in Sudan) as perceived by host and affected communities (i.e., upstream, downstream, and relocated residents), which is not commonly seen in the literature. Primary survey and interviews were conducted with 300 residents, government officials, the Dam Implementation Unit (DIU), NGOs and other key informants. Five inter-related areas of impact were scrutinized: (a) displacement of communities; (b) resettlement of displaced communities in a new location; (c) technological factors; (d) social mobility factors; and (e) economic and political institutions. Results show that Merowe Dam exerted positive as well as adverse social impacts on local c

In [10]:
df['doc_content'] = df['name']+ ' ' + df['doc_title'] +" "+ df['doc_abstract']
df.head()

,name,job_title,organization,doc_link,doc_title,doc_abstract,doc_content
0,Mohamad Nazri Abd Karim,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/stock-price-and-volume-effects-associated-with-changes-in-the-com,Stock Price and Volume Effects Associated with Changes in the Composition of the FTSE Bursa Malaysian KLCI,"We examine the stock price and volume effects of changes in the composition of the FTSE Bursa Malaysia Kuala Lumpur Composite Index (KLCI), over the time period of 2005 to 2012. We find evidence to support the price pressure hypothesis for both additions to and deletions from the KLCI. This is because substantial stock price and trading volume effects in the pre index revision period are entirely reversed after the announcement of the news. Significant changes in liquidity causes trading volume and stock prices to reverse back to their original level before the index revisions took place.","Mohamad Nazri Abd Karim Stock Price and Volume Effects Associated with Changes in the Composition of the FTSE Bursa Malaysian KLCI We examine the stock price and volume effects of changes in the composition of the FTSE Bursa Malaysia Kuala Lumpur Composite Index (KLCI), over the time period of 2005 to 2012. We find evidence to support the price pressure hypothesis for both additions to and deletions from the KLCI. This is because substantial stock price and trading volume effects in the pre index revision period are entirely reversed after the announcement of the news. Significant changes in liquidity causes trading volume and stock prices to reverse back to their original level before the index revisions took place."
1,Muhammad Abdullah,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/assessing-the-use-of-gold-as-a-zero-beta-asset-in-empirical-asset,Assessing the Use of Gold as a Zero-Beta Asset in Empirical Asset Pricing: Application to the US Equity Market,"This paper examines the use of the return on gold instead of treasury bills in empirical asset pricing models for the US equity market. It builds upon previous research on the safe-haven, hedging, and zero-beta characteristics of gold in developed markets and the close relationship between interest rates, stock, and gold returns. In particular, we extend this research by showing that using gold as a zero-beta asset helps to improve the time-series performance of asset pricing models when pricing US equities and industries between 1981 and 2015. The performance of gold zero-beta models is also compared with traditional empirical factor models using the 1-month Treasury bill rate as the risk-free rate. Our results indicate that using gold as a zero-beta asset leads to higher R-squared values, lower Sharpe ratios of alphas, and fewer significant pricing errors in the time-series analysis. Similarly, the pricing of small stock and industry portfolios is improved. In cross-section, we also find improved results, with fewer cross-sectional pricing errors and more economically meaningful pricing of risk factors. We also find that a zero-beta gold factor constructed to be orthogonal to the Carhart four factors is significant in cross-section and helps to improve factor model performance on momentum portfolios. Furthermore, the Fama–French three- and five-factor asset pricing models and the Carhart model are all improved by these means, particularly on test assets which have been poorly priced by the traditional versions. Our results have salient implications for policymakers, governments, central bank rate-setting decisions, and investors.","Muhammad Abdullah Assessing the Use of Gold as a Zero-Beta Asset in Empirical Asset Pricing: Application to the US Equity Market This paper examines the use of the return on gold instead of treasury bills in empirical asset pricing models for the US equity market. It builds upon previous research on the safe

In [11]:
df = df.dropna()
one_word_titles = df[df['doc_title'].str.count('\s') == 0]['doc_title']
df = df[~df['doc_title'].isin(one_word_titles)]
df = df.reset_index()
df = df.drop(columns = 'index')

In [12]:
df_final = df.drop(columns = ['doc_content'])
df_final.head(10)

,name,job_title,organization,doc_link,doc_title,doc_abstract
0,Mohamad Nazri Abd Karim,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/stock-price-and-volume-effects-associated-with-changes-in-the-com,Stock Price and Volume Effects Associated with Changes in the Composition of the FTSE Bursa Malaysian KLCI,"We examine the stock price and volume effects of changes in the composition of the FTSE Bursa Malaysia Kuala Lumpur Composite Index (KLCI), over the time period of 2005 to 2012. We find evidence to support the price pressure hypothesis for both additions to and deletions from the KLCI. This is because substantial stock price and trading volume effects in the pre index revision period are entirely reversed after the announcement of the news. Significant changes in liquidity causes trading volume and stock prices to reverse back to their original level before the index revisions took place."
1,Muhammad Abdullah,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/assessing-the-use-of-gold-as-a-zero-beta-asset-in-empirical-asset,Assessing the Use of Gold as a Zero-Beta Asset in Empirical Asset Pricing: Application to the US Equity Market,"This paper examines the use of the return on gold instead of treasury bills in empirical asset pricing models for the US equity market. It builds upon previous research on the safe-haven, hedging, and zero-beta characteristics of gold in developed markets and the close relationship between interest rates, stock, and gold returns. In particular, we extend this research by showing that using gold as a zero-beta asset helps to improve the time-series performance of asset pricing models when pricing US equities and industries between 1981 and 2015. The performance of gold zero-beta models is also compared with traditional empirical factor models using the 1-month Treasury bill rate as the risk-free rate. Our results indicate that using gold as a zero-beta asset leads to higher R-squared values, lower Sharpe ratios of alphas, and fewer significant pricing errors in the time-series analysis. Similarly, the pricing of small stock and industry portfolios is improved. In cross-section, we also find improved results, with fewer cross-sectional pricing errors and more economically meaningful pricing of risk factors. We also find that a zero-beta gold factor constructed to be orthogonal to the Carhart four factors is significant in cross-section and helps to improve factor model performance on momentum portfolios. Furthermore, the Fama–French three- and five-factor asset pricing models and the Carhart model are all improved by these means, particularly on test assets which have been poorly priced by the traditional versions. Our results have salient implications for policymakers, governments, central bank rate-setting decisions, and investors."
2,Al-Noor Abdullah,Lecturer in Business Management,School of Strategy and Leadership,https://pureportal.coventry.ac.uk/en/publications/social-impacts-of-a-mega-dam-project-as-perceived-by-local-resett,"Social Impacts of a Mega-Dam Project as Perceived by Local, Resettled and Displaced Communities: A Case Study of Merowe Dam, Sudan","The paper assesses social impacts of a mega-dam project (Merowe Dam in Sudan) as perceived by host and affected communities (i.e., upstream, downstream, and relocated residents), which is not commonly seen in the literature. Primary survey and interviews were conducted with 300 residents, government officials, the Dam Implementation Unit (DIU), NGOs and other key informants. Five inter-related areas of impact were scrutinized: (a) displacement of communities; (b) resettlement of displaced communities in a new location; (c) technological factors; (d) social mobility factors; and (e) economic and political institutions. Results show that Merowe Dam exerted positive as well as adverse social impacts on local c

In [13]:
def clean_docs(i, doc):
    stops = stopwords.words('english')
    words = doc.split()

    final = []

    for word in words:
        word = word.lower()
        word = word.replace('-', ' ')
        if word not in stops:
            final.append(word)
        
    final = " ". join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    df_final.loc[i, 'doc_content'] = final

In [14]:
for i in range(0, len(df)):
    data = df.loc[i, 'doc_content']
    clean_docs(i, data)

In [15]:
df_final[0:5]

,name,job_title,organization,doc_link,doc_title,doc_abstract,doc_content
0,Mohamad Nazri Abd Karim,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/stock-price-and-volume-effects-associated-with-changes-in-the-com,Stock Price and Volume Effects Associated with Changes in the Composition of the FTSE Bursa Malaysian KLCI,"We examine the stock price and volume effects of changes in the composition of the FTSE Bursa Malaysia Kuala Lumpur Composite Index (KLCI), over the time period of 2005 to 2012. We find evidence to support the price pressure hypothesis for both additions to and deletions from the KLCI. This is because substantial stock price and trading volume effects in the pre index revision period are entirely reversed after the announcement of the news. Significant changes in liquidity causes trading volume and stock prices to reverse back to their original level before the index revisions took place.",mohamad nazri abd karim stock price volume effects associated changes composition ftse bursa malaysian klci examine stock price volume effects changes composition ftse bursa malaysia kuala lumpur composite index klci time period 2005 2012 find evidence support price pressure hypothesis additions deletions klci substantial stock price trading volume effects pre index revision period entirely reversed announcement news significant changes liquidity causes trading volume stock prices reverse back original level index revisions took place
1,Muhammad Abdullah,Lecturer in Finance,"School of Economics, Finance and Accounting",https://pureportal.coventry.ac.uk/en/publications/assessing-the-use-of-gold-as-a-zero-beta-asset-in-empirical-asset,Assessing the Use of Gold as a Zero-Beta Asset in Empirical Asset Pricing: Application to the US Equity Market,"This paper examines the use of the return on gold instead of treasury bills in empirical asset pricing models for the US equity market. It builds upon previous research on the safe-haven, hedging, and zero-beta characteristics of gold in developed markets and the close relationship between interest rates, stock, and gold returns. In particular, we extend this research by showing that using gold as a zero-beta asset helps to improve the time-series performance of asset pricing models when pricing US equities and industries between 1981 and 2015. The performance of gold zero-beta models is also compared with traditional empirical factor models using the 1-month Treasury bill rate as the risk-free rate. Our results indicate that using gold as a zero-beta asset leads to higher R-squared values, lower Sharpe ratios of alphas, and fewer significant pricing errors in the time-series analysis. Similarly, the pricing of small stock and industry portfolios is improved. In cross-section, we also find improved results, with fewer cross-sectional pricing errors and more economically meaningful pricing of risk factors. We also find that a zero-beta gold factor constructed to be orthogonal to the Carhart four factors is significant in cross-section and helps to improve factor model performance on momentum portfolios. Furthermore, the Fama–French three- and five-factor asset pricing models and the Carhart model are all improved by these means, particularly on test assets which have been poorly priced by the traditional versions. Our results have salient implications for policymakers, governments, central bank rate-setting decisions, and investors.",muhammad abdullah assessing use gold zero beta asset empirical asset pricing application us equity market paper examines use return gold instead treasury bills empirical asset pricing models us equity market builds upon previous research safe haven hedging zero beta characteristics gold developed markets close relationship interest rates stock gold returns particular extend research showing using gold zero beta asset helps improve time series performance asset pricing models pricing us equitie

Exporting clean final df for text processing purposes

In [17]:
df_final.to_csv('cleaned_data.csv', index=False)

In [9]:
documents = list(df_final['doc_content'])

In [10]:
count = 0
for i in range(0, len(documents)):
    count += len(documents[i].split())
    
print(count)

944520


In [11]:
vectorizer = TfidfVectorizer(lowercase = True)

tfidf_matrix = vectorizer.fit_transform(documents)
print(tfidf_matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
print(type(tfidf_matrix), tfidf_matrix.shape)

<class 'scipy.sparse._csr.csr_matrix'> (8924, 48349)


In [13]:

def query_processor(query):
    stops = stopwords.words('english')
    words = query.split()

    final = []

    for word in words:
        word = word.lower()
        word = word.replace('-', ' ')
        if word not in stops:
            final.append(word)
        
    final = " ". join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    return final

In [47]:
query = "covid death"
processed_query = query_processor(query)
#Transform the query using the fitted vectorizer
query_vector = vectorizer.transform([processed_query])

#Compute the cosine similarity
cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)

#Find the most similar document, argsort() causes the max value's index to be at the last 
most_similar_doc_index = cosine_similarities.argsort()[0]
#converting most_similar_doc_index to a list so that the index values can be reversed (in descending order)
similar_index_list = list(most_similar_doc_index)
similar_index_list.reverse()

similarities = []
similarities_percentage = []
most_similar_doc = []
length = int(len(similar_index_list)/(0.1*len(similar_index_list)))

for index in similar_index_list[0:10]:
    similarities.append(cosine_similarities[0][index])
    most_similar_doc.append(documents[index])

for s in similarities:
    similarities_percentage.append((s + 1) / 2 * 100)
   
print("Most similar document: \n", most_similar_doc[0])

Most similar document: 
 jonathan jong religious correlates death anxiety systematic review meta analysis various theories religion hypothesize connection death anxiety religiosity particular terror management theory’s worldview defense hypothesis predicts death anxiety lowest among religious irreligious individuals highest among uncertain individuals likewise supposition death anxiety motivates religious belief turn mitigates death anxiety predicts religiosity increases death anxiety among nonbelievers death anxiety decreases religiosity increases among believers cases curvilinear relationship—specifically inverted u curve—is predicted extracted 202 effect sizes 100 studies “omnibus” religiosity meta analysis six meta analyses examine particular dimensions religiosity found high heterogeneity weak negative association death anxiety religiosity closer examination revealed 10 11 studies directly tested curvilinearity provided support inverted u pattern curvilinearity hypothesis cannot r

In [48]:
most_similar_doc_index

array([   0, 5922, 5920, ..., 3811, 2937, 5160], dtype=int64)

In [49]:
len(most_similar_doc)

10

In [50]:
final_result = []

In [51]:
for i in range(0, len(most_similar_doc)):
    result = df_final.loc[similar_index_list[i]]
#     print(f"Similarity: {similarities[i]}\nS. Percentage: {similarities_percentage[i]:.2f}%\nAuthor: {result['name']}\nDocument Link: {result['doc_link']}")
#     print()
#     print(f"{result['doc_title']}")
#     print()
#     print(f"{result['doc_abstract']}")
#     print("~"*100)
    final_result.append({"Similarity": similarities[i],
                         "Similarity Percentage" : similarities_percentage[i],
                            "Author": result['name'],
                            "Document Link": result['doc_link'],
                            "Title" : result['doc_title'],
                            "Abstract" : result['doc_abstract']
                            })

In [52]:
final_result[0:4]

[{'Similarity': 0.32723440274926946,
  'Similarity Percentage': 66.36172013746348,
  'Author': 'Jonathan Jong',
  'Document Link': 'https://pureportal.coventry.ac.uk/en/publications/the-religious-correlates-of-death-anxiety-a-systematic-review-and-2',
  'Title': 'The religious correlates of death anxiety: A systematic review and meta-analysis',
  'Abstract': 'Various theories of religion hypothesize a connection between death anxiety and religiosity. In particular, Terror Management Theory’s worldview defense hypothesis predicts that death anxiety is lowest among very religious and irreligious individuals, and highest among uncertain individuals. Likewise, the supposition that death anxiety motivates religious belief, which in turn mitigates death anxiety predicts that religiosity increases with death anxiety among nonbelievers, and that death anxiety decreases as religiosity increases among believers. In both cases, a curvilinear relationship—specifically, an inverted-U curve—is predi